<a href="https://colab.research.google.com/github/PravinTiwari023/Pytorch-Tutorial/blob/main/2_Pytorch_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fundation**

Dataset (breast cancer) --> NN--> Training pipeline

---

* Load the dataset
* Basic preprocessing
* Training process:
  1. Create a model
  2. Forward pass
  3. loss calculation
  4. back propagation
  5. Parameters update
* Model evaluation

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

**Train test split**

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

**Scaling**

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Label Encoding**

In [7]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

**Numpy to Pytorch Tensors**

In [8]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [9]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)

torch.Size([455, 30])
torch.Size([455])


**Defining the Model**

In [10]:
class MySimpleNN():

  def __init__(self, X):

    self.weights = torch.rand(X.shape[1], 1, dtype= torch.float64, requires_grad= True)
    self.bias = torch.zeros(1, dtype= torch.float64, requires_grad= True)

  def forward(self, X):

    z = torch.matmul(X, self.weights) + self.bias
    y_pred =  torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):

    # Clamp prediction to avoid log(0) error
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)

    # Calculate the loss
    loss = - (y * torch.log(y_pred) + (1-y) * torch.log(1-y_pred)).mean()
    return loss

**Important Parameters**

In [11]:
learning_rate = 0.1
epochs = 25

**Taining Pipeline**

In [12]:
# Create a model
model = MySimpleNN(X_train_tensor)

# Define loop
for epoch in range(epochs):
  # Forward pass
  y_pred = model.forward(X_train_tensor)

  # Loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)

  # Backward pass
  loss.backward()

  # Parameter update
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # Zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # Print loss in each epoch
  print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.4288875589438224
Epoch: 2, Loss: 3.297287223494617
Epoch: 3, Loss: 3.1623583400616355
Epoch: 4, Loss: 3.0251996659594407
Epoch: 5, Loss: 2.882954849042193
Epoch: 6, Loss: 2.735208583752258
Epoch: 7, Loss: 2.5850171300398057
Epoch: 8, Loss: 2.4284560483068547
Epoch: 9, Loss: 2.2746724539282077
Epoch: 10, Loss: 2.1191523409271076
Epoch: 11, Loss: 1.9605058226283094
Epoch: 12, Loss: 1.8031724287248845
Epoch: 13, Loss: 1.6525949271916351
Epoch: 14, Loss: 1.5095211444890333
Epoch: 15, Loss: 1.3770071881693828
Epoch: 16, Loss: 1.2592500065262884
Epoch: 17, Loss: 1.1575431524513875
Epoch: 18, Loss: 1.0725810459280412
Epoch: 19, Loss: 1.0042303989527945
Epoch: 20, Loss: 0.9513050332822277
Epoch: 21, Loss: 0.9114465871960663
Epoch: 22, Loss: 0.881646603021484
Epoch: 23, Loss: 0.8590910897400155
Epoch: 24, Loss: 0.8415817677407662
Epoch: 25, Loss: 0.8275616619111673


**Evaluation**

In [13]:
# Model Evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy}')

Accuracy: 0.5280086398124695
